#Monte Carlo First Visit Method to solve a simple GridWorld problem.#

Deriving a Greedy Policy from Monte Carlo State-Value Estimation.
<br>In this case, we are using a RANDOM UNIFORM POLICY to train.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys

# where gridworld.py is located
# modify path accordingly
sys.path.append('/content/drive/MyDrive')


In [ ]:
import numpy as np
import gridworld

**Derive the policy to navigate the agent AFTER the agent has been trained through 10000 episodes. In this case, we will choose GREEDY actions that enable the agent to move to the states with the highest value.**

**Standard Grid:**<br>
x means you can't go there<br>
s means start position<br>
number means reward at that state<br>
**.  .  .  1**<br>
**.  x  . -1**<br>
**s  .  .  .**<br>





In [ ]:
# define a grid that describes the reward for arriving at each state and possible actions at each state
# the grid looks like this:
#
# x means you can't go there
# s means start position
# number means reward at that state
#   0  1  2  3
# 0 .  .  .  1
# 1 .  x  . -1
# 2 s  .  .  .
#
# We go by (row, col)
#
MAX_EPISODES = 10000
GAMMA = 0.9
ACTIONS = ['U', 'D', 'L', 'R']

env = gridworld.standard_grid() #3x4 grid
returns = {}  # Dictionary to store returns for each state, returns[s] points to a list
state_values = {} # Dictionary to store state value functions

In [ ]:
#
# Generate Episodes
#
for ep in range(MAX_EPISODES):
	state = env.reset() #return a tuple, state at start
	trajectory = [] #to store trajectory of 1 episode

  #
	# generate 1 episode
  #
	while True:
		# using UNIFORM RANDOM as a policy to train (behavioral policy)
		action = np.random.choice(ACTIONS)

		next_state, reward, done, invalid = env.step(action)
		if invalid: continue #go back and get another random action

		trajectory.append((state, reward)) #St, Rt+1
		state = next_state
		if done: break

  #
	# Analyse the 1 episode generated
	# E.g. trajectory = [((2, 0), 0), ((2, 1), 0), ((2, 0), 0), ((2, 1), 0), ((2, 2), 0), ((1, 2), -1)]
	#
	reverse_trajectory = trajectory[::-1]
	reverse_trajectory_states = list(map(lambda x:x[0], reverse_trajectory)) # just the states only
	G = 0

	#
	# GO THRU trajectory BACKWARD to CALC averages
	# trajectory[] contains (state, reward) of only 1 episode
	#
	for idx, step in enumerate(reverse_trajectory):
		s, r = step
		G = GAMMA*G + r # say, after 3 steps, G=GAMMA*(GAMMA*(GAMMA*G + r3) + r2) + r1
                    #                     G=GAMMA^3*G + GAMMA^2*r3 + GAMMA*r2 + r1
                    #                     G=GAMMA^2*r3 + GAMMA*r2 + r1

    #
    # First-visit MC: only update the FIRST OCCURENCE of the state
    #                         5       4       3       2       1       0
		# E.g. for trajectory = [(2, 0), (2, 1), (2, 0), (2, 1), (2, 2), (1, 2)]
		#      it will be idx: 0, 1, 4, 5
		if s not in reverse_trajectory_states[idx+1:]:
			# we add state s to dict returns, returns[s] points to a list
			if s in returns: returns[s].append(G) # returns is a dict that points to lists
			else:            returns[s] = [G] # first guy in the list

			# update state_values[s] dict, which stores mean gain results of each state s
			state_values[s] = np.mean(returns[s]) # returns[s] points to a list

    # The terminal states will have 0 value cause no further rewards are received after reaching them.
    # That is, agent doesn't move from terminal states to get further rewards. That is why we
    # manually insert the 2 final values (rewards) in the terminal states
		state_values[(0,3)]=1
		state_values[(1,3)]=-1

print("Training complete.")

Training complete.


In [ ]:
# Define function to derive policy
def choose_action(state):
    """
    Given a state, choose the best action based on learned state values.
    Uses a greedy policy to select the action that leads to the state with the highest value.

    Parameters:
    state (tuple): The current state (row, col) in the grid.

    Returns:
    str: The best action ('U', 'D', 'L', 'R') or 'X' for terminal states.
    """

    # If the state is a terminal state, return 'X' (no action needed)
    if state in env.terminal_states:
        return 'X'

    # Check if the state has available actions
    if state in env.actions:
        possible_actions = env.actions[state]  # Get possible actions
        best_action = None  # Track the best action
        best_value = 0.0  # Initialize the best value to 0

        # Iterate over all possible actions to find the best one
        for action in possible_actions:
            env.reset()  # Reset environment to the given state
            env.i, env.j = state  # Set the agent to the current state
            next_state, _, _, invalid = env.step(action)  # Take action and observe next state, ignoring reward and done status.

            if invalid:
                continue  # Skip invalid moves

            value = state_values.get(next_state, 0.0)  # Get state value, default to 0
            if value > best_value:  # If this state has a higher value, update best action
                best_value = value
                best_action = action

        # Return the best action, or a random valid action if all values are equal
        return best_action if best_action else possible_actions[0]  # Always choose first action

    return ' '  # Return empty space if no valid actions exist

In [ ]:
print("State Values:")
gridworld.print_values(state_values, env)

# Derive policy
greedy_policy = {}
for state in env.all_states():
    greedy_policy[state] = choose_action(state)

print("\nDerived Optimal Policy:")
gridworld.print_policy(greedy_policy, env)

State Values:
---------------------------
 0.05| 0.14| 0.27| 1.00|
---------------------------
-0.02| 0.00|-0.35|-1.00|
---------------------------
-0.10|-0.21|-0.37|-0.66|

Derived Optimal Policy:
---------------------------
  R  |  R  |  R  |  X  |
---------------------------
  U  |     |  U  |  X  |
---------------------------
  U  |  L  |  L  |  L  |
